# The Milwaukee Journal Sentinel and Life Expectancy

**Story:** [Race gap found in pothole patching](https://web.archive.org/web/20081223094123/http://www.jsonline.com/news/milwaukee/32580034.html)

**Author:** Keegan Kyle, Grant Smith and Ben Poston, Milwaukee Journal Sentinel

**Topics:** Census Data, Geocoding, QGIS Spatial Joins, Linear Regression

**Datasets**

* Pothole fill data from the Milwaukee [Department of Public Works](https://city.milwaukee.gov/dpw)
    - **2007-2010 POTHOLES.xls**: Pothole data, July 2007-July 2010
    - **2010-2013 POTHOLES.xls**: Pothole data, July 2010-July 2013
    - **2013-2017 POTHOLES.xls**: Pothole data, July 2013-July 2017
* **tl_2013_55_tract.zip:** 2013 census tract boundaries from the [US Census Bureau](https://www.census.gov/cgi-bin/geo/shapefiles/index.php)
* **addresses_geocoded.csv:** a large selection of addresses in Milwaukee, geocoded by [Geocod.io](https://geocod.io)
* **R12216099_SL140.csv:** ACS 2013 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table A04001, Hispanic or Latino by Race
    - `R12216099.txt` is the data dictionary
* **R12216226_SL140.csv** ACS 2013 5-year, tract level, from [Social Explorer](https://www.socialexplorer.com)
    - Table A14006, 2013 Median Household income
    - Data dictionary [is here](https://www.socialexplorer.com/data/ACS2013_5yr/metadata/?ds=SE&table=A14006)

# What's the story?

We're trying to figure out if the **time it took Milwaukee to fill pot holes** is related to the racial makeup of a census tract.

# Do your imports

You'll also want to set pandas to display **up to 200 columns at a time**.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import requests as re
pd.set_option('display.max_columns', 200)

# Create a dataframe of potholes from only 2013

The delivered multiple files to us, so we'll need to filter them down to just 2013.

**Editorial decision:** When we say "a pothole from 2013," what exactly do we mean?

In [4]:
df1 = pd.read_excel('2010-2013 POTHOLES.xls')
df2 = pd.read_excel('2013-2017 POTHOLES.xls')

In [5]:
df1.head()

,A,Street,EnterDt,PrintDt,ResolvDt
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08


In [6]:
df1 = df1[df1.EnterDt.str.contains('2013', regex=True)|df1.ResolvDt.str.contains('2013', regex=True)]

In [7]:
df2 = df2[df2.EnterDt.str.contains('2013', regex=True)|df2.ResolvDt.str.contains('2013', regex=True)]

#Or str.startswith('2013')

In [8]:
df_2013 = df1.append(df2)
df_2013

,A,Street,EnterDt,PrintDt,ResolvDt
0,3839,N 10TH ST,2013-07-15 23:35,2013-07-16 05:46,2013-07-17 05:50
1,4900,W MELVINA ST,2013-07-15 20:05,2013-07-16 05:46,2013-07-24 16:58
2,2400,W WISCONSIN AV,2013-07-15 20:00,2013-07-16 05:56,2013-07-25 14:42
3,1800,W HAMPTON AV,2013-07-15 19:55,2013-07-16 05:46,2013-07-18 06:06
4,4718,N 19TH ST,2013-07-15 19:50,2013-07-16 05:46,2013-08-02 06:08
5,4500,W CLARKE ST,2013-07-15 18:05,2013-07-16 05:56,2013-08-01 14:41
6,6471,N 66TH ST,2013-07-15 17:44,2013-07-16 05:46,2013-08-14 05:58
7,6200,W BALDWIN ST,2013-07-15 17:35,2013-07-16 05:46,2013-08-02 06:09
8,3013,N 74TH ST,2013-07-15 17:23,2013-07-16 05:46,2013-07-16 05:49
9,2606,N 7TH ST,2013-07-15 17:13,2013-07-16 05:55,2013-08-07 06:06


## What is the maximum and minimum `EnterDt` and `ResolvDt`?

Use this to confirm that your date range is what you expected. If it isn't, take a look at what might have happened with your dataset.

* **Tip:** Missing data might be a headache

In [9]:
df_final = df_2013.dropna()

In [10]:
df_final.EnterDt.max()
#or df_final.EnterDt.agg(['min','max'])

'2013-12-31 17:06'

In [11]:
df_final.EnterDt.min()

'2012-12-17 10:10'

In [12]:
df_final.ResolvDt.max()

'2014-01-23 17:05'

In [13]:
df_final.ResolvDt.min()

'2013-01-02 06:13'

## Calculate how long it took to fill potholes in 2013

Save it into a new column.

* **Tip:** It's possible to subtract two dates

In [14]:
pd.to_datetime(df_final['EnterDt'])

0       2013-07-15 23:35:00
1       2013-07-15 20:05:00
2       2013-07-15 20:00:00
3       2013-07-15 19:55:00
4       2013-07-15 19:50:00
5       2013-07-15 18:05:00
6       2013-07-15 17:44:00
7       2013-07-15 17:35:00
8       2013-07-15 17:23:00
9       2013-07-15 17:13:00
10      2013-07-15 16:55:00
11      2013-07-15 16:35:00
12      2013-07-15 16:30:00
13      2013-07-15 16:10:00
14      2013-07-15 16:14:00
15      2013-07-15 15:50:00
16      2013-07-15 16:15:00
17      2013-07-15 15:50:00
18      2013-07-15 15:26:00
19      2013-07-15 14:15:00
20      2013-07-15 13:05:00
21      2013-07-15 12:55:00
22      2013-07-15 13:09:00
23      2013-07-15 12:30:00
24      2013-07-15 12:20:00
25      2013-07-15 12:15:00
26      2013-07-15 12:15:00
27      2013-07-15 14:29:00
28      2013-07-15 12:00:00
29      2013-07-15 11:57:00
                ...        
48397   2013-07-16 12:20:00
48398   2013-07-16 12:10:00
48399   2013-07-16 11:55:00
48400   2013-07-16 11:53:00
48401   2013-07-16 1

In [15]:
df_final['ResolvDt']=pd.to_datetime(df_final['ResolvDt'])

/Users/giovanafleck/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
df_final['EnterDt']=pd.to_datetime(df_final['EnterDt'])

/Users/giovanafleck/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_final['difference']=df_final['ResolvDt']-df_final['EnterDt']

/Users/giovanafleck/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_final['difference'].mean()

Timedelta('9 days 04:20:37.791946')

In [19]:
df_final.head()

,A,Street,EnterDt,PrintDt,ResolvDt,difference
0,3839,N 10TH ST,2013-07-15 23:35:00,2013-07-16 05:46,2013-07-17 05:50:00,1 days 06:15:00
1,4900,W MELVINA ST,2013-07-15 20:05:00,2013-07-16 05:46,2013-07-24 16:58:00,8 days 20:53:00
2,2400,W WISCONSIN AV,2013-07-15 20:00:00,2013-07-16 05:56,2013-07-25 14:42:00,9 days 18:42:00
3,1800,W HAMPTON AV,2013-07-15 19:55:00,2013-07-16 05:46,2013-07-18 06:06:00,2 days 10:11:00
4,4718,N 19TH ST,2013-07-15 19:50:00,2013-07-16 05:46,2013-08-02 06:08:00,17 days 10:18:00


### Hrm, well, I think we need that difference to be an integer

If your new column isn't an integer, create _another_ column that is.

* **Tip:** Just like you might use `.str.strip()` on a string column, if your column is a datetime you can use `.dt.components` to get the days, hours, minutes, seconds, etc of the column.

In [27]:
df_final['difference'].dt.components

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,1,6,15,0,0,0,0
1,8,20,53,0,0,0,0
2,9,18,42,0,0,0,0
3,2,10,11,0,0,0,0
4,17,10,18,0,0,0,0
5,16,20,36,0,0,0,0
6,29,12,14,0,0,0,0
7,17,12,34,0,0,0,0
8,0,12,26,0,0,0,0
9,22,12,53,0,0,0,0


# Read in your race-related census data

* **Tip:** This will probably give you an error the first time you try, because the Census Bureau doesn't think about the rest of the world.

In [32]:
df_cesus = pd.read_csv('R12216099_SL140.csv', encoding ='latin-1', dtype={'Geo_FIPS':'str'})

In [33]:
df_cesus.head()

,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,Geo_STATECE,Geo_STATE,Geo_COUNTY,Geo_COUSUB,Geo_PLACE,Geo_PLACESE,Geo_TRACT,Geo_BLKGRP,Geo_CONCIT,Geo_AIANHH,Geo_AIANHHFP,Geo_AIHHTLI,Geo_AITSCE,Geo_AITS,Geo_ANRC,Geo_CBSA,Geo_CSA,Geo_METDIV,Geo_MACC,Geo_MEMI,Geo_NECTA,Geo_CNECTA,Geo_NECTADIV,Geo_UA,Geo_UACP,Geo_CDCURR,Geo_SLDU,Geo_SLDL,Geo_VTD,Geo_ZCTA3,Geo_ZCTA5,Geo_SUBMCD,Geo_SDELM,Geo_SDSEC,Geo_SDUNI,Geo_UR,Geo_PCI,Geo_TAZ,Geo_UGA,Geo_BTTR,Geo_BTBG,Geo_PUMA5,Geo_PUMA1,SE_A04001_001,SE_A04001_002,SE_A04001_003,SE_A04001_004,SE_A04001_005,SE_A04001_006,SE_A04001_007,SE_A04001_008,SE_A04001_009,SE_A04001_010,SE_A04001_011,SE_A04001_012,SE_A04001_013,SE_A04001_014,SE_A04001_015,SE_A04001_016,SE_A04001_017
0,55079000101,14000US55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin","Census Tract 1.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4717,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5306,5015,1343,3567,13,70,0,16,6,291,37,0,0,0,0,254,0
1,55079000102,14000US55079000102,"Census Tract 1.02, Milwaukee County, Wisconsin","Census Tract 1.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4718,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3648,3549,1034,2195,36,118,0,0,166,99,67,32,0,0,0,0,0
2,55079000201,14000US55079000201,"Census Tract 2.01, Milwaukee County, Wisconsin","Census Tract 2.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4719,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4791,4019,1185,2657,0,102,0,0,75,772,120,0,0,0,0,613,39
3,55079000202,14000US55079000202,"Census Tract 2.02, Milwaukee County, Wisconsin","Census Tract 2.02, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4720,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6324,6148,2687,2810,87,390,12,0,162,176,113,0,0,0,0,63,0
4,55079000301,14000US55079000301,"Census Tract 3.01, Milwaukee County, Wisconsin","Census Tract 3.01, Milwaukee County, Wisconsin",wi,140,0,ACSSF,4721,NaN,NaN,NaN,NaN,55,79,NaN,NaN,NaN,301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339,1339,1005,220,0,90,0,10,14,0,0,0,0,0,0,0,0


## Create new dataframe describing the racial makeup of each census tract

Each row should contain:

* The census tract number
* The percent of the population that is White
* The percent of the population that is Black
* The percent of the population that is Hispanic
* The percent of the population that is a minority (non-White)

You can either calculate and filter on the original shapefile, or create a whole new dataframe.

* **Tip:** You'll want to read the data dictionary for the Census data.
* **Tip:** The column names don't exactly match the data dictionary, but you can figure it out
* **Tip:** If you aren't sure which column is the actual census tract number, you can keep multiples for now

In [34]:
race_df = pd.read_csv('R12216099_SL140.csv')

race_df2 = race_df[['Geo_TRACT','SE_A04001_001', 'SE_A04001_002',
       'SE_A04001_003', 'SE_A04001_004', 'SE_A04001_005', 'SE_A04001_006',
       'SE_A04001_007', 'SE_A04001_008', 'SE_A04001_009', 'SE_A04001_010',
       'SE_A04001_011', 'SE_A04001_012', 'SE_A04001_013', 'SE_A04001_014',
       'SE_A04001_015', 'SE_A04001_016', 'SE_A04001_017']]
race_df2.rename(columns={'Geo_TRACT':'census tract','SE_A04001_001':'Total Population', 
                        'SE_A04001_002':'Not Hispanic or Latino',
                        'SE_A04001_003':'Not Hispanic or Latino: White Alone', 
                        'SE_A04001_004':'Not Hispanic or Latino: Black or African American Alone', 
                        'SE_A04001_005':'Not Hispanic or Latino: American Indian and Alaska Native Alone', 
                        'SE_A04001_006':'Not Hispanic or Latino: Asian Alone',
                        'SE_A04001_007':'Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone', 
                        'SE_A04001_008':'Not Hispanic or Latino: Some Other Race Alone', 
                        'SE_A04001_009':'Not Hispanic or Latino: Two or More Races', 
                        'SE_A04001_010':'Hispanic or Latino',
                        'SE_A04001_011':'Hispanic or Latino: White Alone', 
                        'SE_A04001_012':'Hispanic or Latino: Black or African American ', 
                        'SE_A04001_013':'Hispanic or Latino: American Indian and Alaska Native Alone', 
                        'SE_A04001_014':'Hispanic or Latino: Asian Alone',
                        'SE_A04001_015':'Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone', 
                        'SE_A04001_016':'Hispanic or Latino: Some Other Race Alone', 
                        'SE_A04001_017':'Hispanic or Latino: Two or More Races'})

,census tract,Total Population,Not Hispanic or Latino,Not Hispanic or Latino: White Alone,Not Hispanic or Latino: Black or African American Alone,Not Hispanic or Latino: American Indian and Alaska Native Alone,Not Hispanic or Latino: Asian Alone,Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone,Not Hispanic or Latino: Some Other Race Alone,Not Hispanic or Latino: Two or More Races,Hispanic or Latino,Hispanic or Latino: White Alone,Hispanic or Latino: Black or African American,Hispanic or Latino: American Indian and Alaska Native Alone,Hispanic or Latino: Asian Alone,Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone,Hispanic or Latino: Some Other Race Alone,Hispanic or Latino: Two or More Races
0,101,5306,5015,1343,3567,13,70,0,16,6,291,37,0,0,0,0,254,0
1,102,3648,3549,1034,2195,36,118,0,0,166,99,67,32,0,0,0,0,0
2,201,4791,4019,1185,2657,0,102,0,0,75,772,120,0,0,0,0,613,39
3,202,6324,6148,2687,2810,87,390,12,0,162,176,113,0,0,0,0,63,0
4,301,1339,1339,1005,220,0,90,0,10,14,0,0,0,0,0,0,0,0
5,302,3091,2946,448,2363,11,40,0,0,84,145,74,25,0,0,0,34,12
6,303,2029,1980,864,941,0,77,0,0,98,49,25,0,0,0,0,24,0
7,304,3072,3024,1318,1140,0,445,0,38,83,48,36,0,0,0,0,12,0
8,400,2450,2365,610,1384,79,0,0,0,292,85,0,0,0,0,0,85,0
9,501,3539,3348,1576,1436,0,62,0,27,247,191,45,0,0,0,0,37,109


# Combine this census data with your street address data

Create a new dataframe my merging your street addresses with your census data. It would be nice to marge on census tract code, but your addresses data does **not** have a census tract on them! **This is where things get a little crazy.**

### Adding census tract codes to addresses

We went ahead and geocoded many addresses in Milwaukee to create `addresses_geocoded.csv`. You'll need to add a new column using QGIS that adds in the address's census tract.

The QGIS command **Join attributes by location** will merge datasets that geographically overlap each other. If you have one layer that's a list of lat/lon points and one layer that's a shapefile with census tract information, **join attributes by location** can create a new layer of lat/lon points that also has census tract information.

You can export the result by right clicking the layer, selecting **Export As** and making sure the output is another CSV.

### Merging

Now you'll have three datasets:

* Addresses and pothole fill ties
* Census data according with census tract codes
* Addresses with census tract codes

You'll need to merge them each together to create one whole dataset.

# Linear regression

Using the `statsmodels` package, run a linear regression to find the coefficient relating percent minority and pothole fill times.

Translate that into the form **"every X percentage point change in the minority population translates to a Y change in pot hole fill times"**

Do you feel comfortable that someone can understand that? Can you reword it to make it more easily understandable?

# Other methods of explanation

While the regression is technically correct, it just does't sound very nice. What other options do we have?

## What's the average wait to fill a pothole between majority-white and majority-minority census tracts?

You'll need to create a new column to specify whether the census tract is majority White or not.

## How does the average wait time to fill a pothole change as more minorities live in an area?

* **Tip:** Use `.cut` or `.qcut` to split the percent minority (or white) into a few different bins.

# Bonus: Income

`R12216226_SL140.csv` contains income data for each census tract in Wisconsin. Add it into your analysis.

If you run a multivariate regression also including income, how does this change things?